In [ ]:
#export
import torch
import torch.nn as nn
from fastai.vision.all import *
from fastai.data.all import *
from fastai.distributed import *
import pandas as pd
from pathlib import Path
import time
from video_block import * 
from inflator import *
from triplet_loss import *

ModuleNotFoundError: No module named 'video_block'

In [ ]:
import torch
import torch.nn as nn
from fastai.vision.all import *
from fastai.data.all import *
from fastai.distributed import *
import pandas as pd
from pathlib import Path
import time
from modules_th.video_block import * 
from modules_th.inflator import *
from modules_th.triplet_loss import *

In [ ]:
class NullModel(Module):
    def __init__(self): pass
    def forward(*x, **kwargs): return x

class NullLoss(Module):
    def __init__(self): pass
    def forward(*x, **kwargs): return torch.tensor(0.,requires_grad=True)
    
def null_splitter(m): return [[],[]]

In [ ]:
#export
def read_data():
    path_data = '/mnt/data/adrianlopez/Videos/Charades/df.csv'
    df = pd.read_csv(path_data, index_col=0).dropna()
    return df 

In [ ]:
df = read_data()

In [ ]:
#export
def get_datasets(df, l, n_lbl, n_el):
    def get_vid_path(df:pd.Series): return Video(df['paths'].split('\n'))
    def get_lbls(df:pd.Series): return df['lbl']

    tfms = [[get_vid_path, ResizeTime(l=l), Video.create],
            [get_lbls, Categorize()]]

    splits = RandomSplitter(valid_pct=0.2, seed=42)(df)
    splits = [uniformize_dataset(idxs, df.iloc[idxs]['lbl'] , n_lbl=n_lbl, n_el=n_el) for idxs in splits]

    dsets = Datasets(df, tfms, splits=splits)
    return dsets

In [ ]:
ds = get_datasets(df, l=20, n_lbl=4, n_el=8)
for i in range(-1,-10,-1): print(ds.valid[i][1])

TensorCategory(74)
TensorCategory(74)
TensorCategory(74)
TensorCategory(74)
TensorCategory(74)
TensorCategory(74)
TensorCategory(74)
TensorCategory(74)
TensorCategory(24)


In [ ]:
#export
def get_dls(df, sz, l, n_lbl, n_el):
    dsets = get_datasets(df, l, n_lbl, n_el)
    dls = dsets.dataloaders(bs=n_el*n_lbl,
                            device=torch.device('cuda:1'),
                            shuffle_train=False,
                            after_item=[Resize(sz), ToTensor()],
                            after_batch=[IntToFloatTensor(), Normalize.from_stats(*imagenet_stats)])
    return dls

In [ ]:
dls = get_dls(df, 224, 20, 4, 8)
dls.one_batch()

NameError: name 'get_datasets' is not defined

In [ ]:
#export
def get_learner(df, l, sz, n_lbl, n_el):
    dls = get_dls(df, sz, l, n_lbl, n_el)   
    head, body = inflate(create_head(1024, len(dls.vocab), lin_ftrs=[256])), inflate(create_body(resnet34, cut=-2)) 
    learn = Learner(dls, 
    #                 NullModel(),
                    TLModel(body,head),
    #                 loss_func=NullLoss(),
                    MixedLoss(alpha=1.0, margin=.2),
    #                 splitter=null_splitter,
                    splitter=my_splitter,
    #                 opt=Optimizer([0], noop),
    #                 metrics=lambda *x: 0)
                    metrics=tl_accuracy)
    return learn

In [ ]:
df = read_data()
learn = get_learner(df, l=10, sz=128, n_lbl=4, n_el=8)

In [ ]:
#exporti
df = read_data()
learn = get_learner(df, l=10, sz=128, n_lbl=4, n_el=8)
learn.fine_tune(1)

FileNotFoundError: [Errno 2] No such file or directory: 'videodata/Charades_v1_rgb/TK00O/TK00O-000021.jpg'

In [ ]:
with learn.parallel_ctx(device_ids=[0,1]):
    learn.fine_tune(4)

KeyboardInterrupt: 

In [ ]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_triplet_loss.ipynb.
Converted 02_inflator.ipynb.
Converted 03_video_block.ipynb.
Converted 04_charades.ipynb.
Converted 05_ucf.ipynb.
Converted 06_distributed_inflated_NN.ipynb.
Converted 07_read_video_function.ipynb.
Converted 08_transformer.ipynb.
Converted index.ipynb.


In [ ]:
# default_exp charades